Step 1: Format Excel file into a GOOD file:
- Copy/paste values into spreadsheet(make sure columns match(NO POSTMATES) & no extra values ANYWHERE)
- The Debit/Credit column highlights in red any imbalances which NEEDS TO BE fixed.
- Once all the debit/credits are balanced, SAVE excel file, press anywhere on box below and press ctrl+enter.
- Continue to Step 2.

In [6]:
import pandas as pd
#fname is name of excel file
fname= "BK Master TEMPLATE.xlsx"
df= pd.DataFrame(pd.read_excel(fname, header=1))
df["Date"] = df["Date"].apply(lambda x: x.strftime("%m/%d/%Y"))

def excel_file_checker(dff):
    if sum(dff.loc[:,"Difference"])==0:
        if len(pd.unique(dff.loc[:,"Store #"])) == 1:
            check = "All good!"
            return check
        else:
            check = "More than 1 store in excel sheet \n Did you copy&paste correctly?"
            return check
    else:
        check = "Excel file needs to be fixed.Check the right-most \"Difference\" column.\nDid you SAVE the file?"
        return check

print(df.iloc[0,0])
print(excel_file_checker(df))

Burger King #24045  
All good!


Step 2: Run 1st box:
- Press anywhere on box below and press ctrl+enter

In [7]:
import pyautogui as pag
import numpy as np 
import datetime

################################################################################################

#single/multiple day entry functions
def day_entry(day_n):

    #Columns for debit are 3-13 on Excel
    dbt_list = np.round(list(df.iloc[day_n-1,2:13]),2)
    #Columns for credit are 14-21 on Excel
    crd_list = np.round(list(df.iloc[day_n-1,13:21]),2)

    dbt_total = np.round(np.sum(np.round(dbt_list,2)),2)
    crd_total = np.round(np.sum(np.round(crd_list,2)),2)

    dbt_crdt_diff = np.round(dbt_total - crd_total,2)

    #1c adjustments are made to SICOM
    if dbt_crdt_diff == -0.01: #Dbt is less than crd by 1c
        dbt_list[10] += 0.01
        dbt_total = np.round(np.sum(np.round(dbt_list,2)),2)
    elif dbt_crdt_diff == 0.01: #Dbt is greater than crd by 1c
        dbt_list[10] -= 0.01
        crd_total = np.round(np.sum(np.round(crd_list,2)),2)

    def input_date_to_first_entry(day): 
        #For date, then we get to first entry
        x=70
        y=220
        pag.moveTo(x, y,duration=0.20)
        pag.doubleClick()
        #twice bc of bug when clicked on somewhere else on QBs
        pag.moveTo(x, y,duration=0.20)
        pag.doubleClick()
        pag.typewrite(df.loc[day_n-1,"Date"])

        # to go from date to 1st entry
        for i in range(4):
            pag.hotkey('tab')

    def auto_entry():
        #data entry
        for d in dbt_list:
            pag.typewrite(str(np.round(d,2)))
            pag.hotkey('down')
            # ex.) df.iloc[0,2] should return 1st day cash deposit
        for c in crd_list:
            pag.typewrite(str(-1*c))
            pag.hotkey('down')

    input_date_to_first_entry(day_n)
    auto_entry()

################################################################################################
def dns_entries(begin_day,end_day):

    #To go to/click Daily sales-New
    x= 70
    y= 168
    pag.moveTo(x, y,duration=0.1)
    pag.doubleClick()

    for i in range(begin_day,end_day+1):
        
        day_entry(i)

        #Save & Close
        x=670
        y=1010
        pag.moveTo(x, y,duration=0.15)
        pag.click()

        #To Daily New Sales
        x= 70
        y= 160
        pag.moveTo(x, y,duration=0.30)
        pag.doubleClick()
        # pag.moveTo(x, y,duration=0)

Step 3: Automate Entries:
- Go to Lists-> Memorized Transaction List -> Full screen(on leftside of code)
- Make sure the day range in dns_entries(1st day, 2nd day) is correct!!! (otherwise you'll make duplicate entries)
    - ex.) For Feb. it would be dns_entries(1,28)
- Click anywhere on box below, press ctrl+enter, DO NOT touch mouse or keyboard while it's working.
    - MOVE MOUSE TO ANY CORNER TO QUIT PROGRAM WHEN WINDOW APPEARS
    - BALANCE, then save & new and then put NEXT DAY to end day:
        - dns_entries(NEXT DAY, end day)    ex.) dns_entries(12,31)


In [ ]:
dns_entries(11,31)

Step 4: When things don't go smoothly.....
- If there is an imbalance of more than $0.01(which should've been fixed in step 1. A +/-$0.01 will be subtracted from SICOM), QBs will make a pop-up window show, so you need to STOP CODE by dragging mouse to any corner of screen.
- Once you resolve the imbalance for that specific date -> Save & Close
- CHANGE dns_entries's 1st day value to have the NEXT DAY.
- ex.) dns_entries(1,31) pauses at 5th day, you fix the entry, then change code to dns_entries(6,31), run code.